# ExecTimer

In [1]:
import numpy as np
import time


import sys
sys.path.append('..')

from astroLuSt.monitoring import timers

import importlib
importlib.reload(timers)

<module 'astroLuSt.monitoring.timers' from 'c:\\Users\\lukas\\Documents\\GitHub\\astroLuSt\\demos\\..\\astroLuSt\\monitoring\\timers.py'>

## Standard Usage

In [17]:
ET = timers.ExecTimer(verbose=1)

ET.checkpoint_start('Sup Task', comment='Global Task')

ET.checkpoint_start('Sub Task1')
time.sleep(2)
ET.checkpoint_end('Sub Task1')
ET.checkpoint_start('Sub Task2')
time.sleep(3)
ET.checkpoint_end('Sub Task2')

ET.checkpoint_end('Sup Task')

ET.estimate_runtime('Sup Task', nrepeats=4, ndone=1)

ET.df_protocoll



######################################################################
INFO: Started Sup Task at 2023-03-17T15:07:06

######################################################################
INFO: Started Sub Task1 at 2023-03-17T15:07:06

INFO: Finished Sub Task1 at 2023-03-17T15:07:08
Required time: 0 days 00:00:02
######################################################################

######################################################################
INFO: Started Sub Task2 at 2023-03-17T15:07:08

INFO: Finished Sub Task2 at 2023-03-17T15:07:11
Required time: 0 days 00:00:03
######################################################################

INFO: Finished Sup Task at 2023-03-17T15:07:11
Required time: 0 days 00:00:05
######################################################################
INFO: Total estimated runtime for 4 repeats: 0 days 00:00:20


,Task,Start,End,Duration,Comment
0,Sup Task,2023-03-17 15:07:06,2023-03-17T15:07:11,0 days 00:00:05,Global Task
1,Sub Task1,2023-03-17 15:07:06,2023-03-17T15:07:08,0 days 00:00:02,
2,Sub Task2,2023-03-17 15:07:08,2023-03-17T15:07:11,0 days 00:00:03,


## Usage to have an up-to-date estimate of runtimes

In [16]:
importlib.reload(timers)

#initialize the timer
ET = timers.ExecTimer(verbose=0)

repeats = 5 #number of times to repeat the loop


#Execute some loop
ET.checkpoint_start('Global Task')  #to monitor total (actual) duration
for idx in range(repeats):

    #monitor duration of each loop-execution
    ET.checkpoint_start(f'Subtask{idx}')

    #generate random sleeptime (should average out to be 1)
    time.sleep(np.random.normal(1, 0.5, size=1)[0])
    
    ET.checkpoint_end(f'Subtask{idx}')

    #estimate the runtime from the current point of view
    ET.estimate_runtime(r'Subtask', repeats, idx+1)

ET.checkpoint_end('Global Task')

ET.df_protocoll

INFO: Total estimated runtime for 5 repeats: 0 days 00:00:00
INFO: Total estimated runtime for 5 repeats: 0 days 00:00:02.500000
INFO: Total estimated runtime for 5 repeats: 0 days 00:00:01.666666666
INFO: Total estimated runtime for 5 repeats: 0 days 00:00:02.500000
INFO: Total estimated runtime for 5 repeats: 0 days 00:00:03


,Task,Start,End,Duration,Comment
0,Global Task,2023-03-17 15:06:43,2023-03-17T15:06:46,0 days 00:00:03,
1,Subtask0,2023-03-17 15:06:43,2023-03-17T15:06:43,0 days 00:00:00,
2,Subtask1,2023-03-17 15:06:43,2023-03-17T15:06:44,0 days 00:00:01,
3,Subtask2,2023-03-17 15:06:44,2023-03-17T15:06:44,0 days 00:00:00,
4,Subtask3,2023-03-17 15:06:44,2023-03-17T15:06:45,0 days 00:00:01,
5,Subtask4,2023-03-17 15:06:45,2023-03-17T15:06:46,0 days 00:00:01,
